### Enviroment prepare (imports)

In [2]:
import matplotlib.pyplot as plt
import torch
import numpy as np
import torch.nn as nn
import torchvision
import torch.nn.functional as F
from torch.optim import Adam
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import  DataLoader,SubsetRandomSampler,ConcatDataset,Subset
from sklearn.model_selection import KFold
import pandas as pd

## DATA PREPARE

###### Here we'll use a affectnet net as training and final test test validation to our CNN model, where we're going to test 3 principal models: VGG, RESNET AlexNet. As validation, the FER-2013 dataset will be splited and cross validate with the training data to have a more robust trainig.


### IMPORT DATA

#### Transformation functions and hyper variables

###### Here we opted to use a 64x64px image proportion, because face images usually have a smaller size in pictures when we extract.

In [3]:
batch_size = 64
classes = ('angry', 'fear', 'happy', 'neutral', 'sad', 'surprise','disgust')

train_transform = transforms.Compose([
    # transforms.Resize((64,64)),
    transforms.Resize((224,224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

In [4]:
train_dataset = datasets.ImageFolder('./fer_ckplus_dataset', transform=train_transform)
train_loaded = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = datasets.ImageFolder('./color_dataset_2/train', transform=train_transform)

## Cross-Validation 

###### We'll use the cross validation with a diferent dataset, to improve the cnn knowledge

In [5]:
k=10
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

## Define Models Architecture

#### AlexNet model to 64x64px

In [6]:
class FacialExpressionAlexNet(nn.Module):
    def __init__(self, num_classes=7):
        super(FacialExpressionAlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(256, 384, kernel_size=3, padding=1),

            nn.Conv2d(384, 384, kernel_size=3, padding=1),

            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))

        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),

            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),

            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
class VGG19(nn.Module):
    def __init__(self, num_classes=7):
        super(VGG19, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### Model Definition

In [7]:
model = VGG19()

### Define loss function and optimizer

In [8]:
# Define your execution device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Runing on: "+ ("cuda" if torch.cuda.is_available() else "cpu"))

learning_rate = 0.001

loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=learning_rate,  weight_decay = 0.001)

Runing on: cuda


### Model save

In [9]:
def saveModel():
    torch.save(model.state_dict(), "apurated_model.pth")

## Training and Validation functions

In [10]:
def train_epoch(model,device,dataloader):
    train_loss,train_correct=0.0,0
    model.train()
    for images, labels in dataloader:

        images,labels = images.to(device),labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * images.size(0)
        scores, predictions = torch.max(output.data, 1)
        train_correct += (predictions == labels).sum().item()

    return train_loss,train_correct
  
def valid_epoch(model,device,dataloader):
    valid_loss, val_correct = 0.0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:

            images,labels = images.to(device),labels.to(device)
            output = model(images)
            loss=loss_fn(output,labels)
            valid_loss+=loss.item()*images.size(0)
            scores, predictions = torch.max(output.data,1)
            val_correct+=(predictions == labels).sum().item()

    return valid_loss,val_correct

In [11]:
def train(num_epochs):
    history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
    best_accuracy = 0.0

    model.to(device)
    
    for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(validation_dataset)))):

        print('Fold {}'.format(fold + 1))

        test_sampler = SubsetRandomSampler(val_idx)
        test_loader = DataLoader(validation_dataset, batch_size=batch_size, sampler=test_sampler)
        

        for epoch in range(num_epochs):
            train_loss, train_correct=train_epoch(model,device,train_loaded)
            test_loss, test_correct=valid_epoch(model,device,test_loader)

            train_loss = train_loss / len(train_loaded.sampler)
            train_acc = train_correct / len(train_loaded.sampler) * 100

            test_loss = test_loss / len(test_loader.sampler)
            test_acc = test_correct / len(test_loader.sampler) * 100

            print("Epoch:{}/{} AVG Training Loss:{:.3f} AVG Test Loss:{:.3f} AVG Training Acc {:.2f} % AVG Test Acc {:.2f} %".format(epoch + 1,
                                                                                                                    num_epochs,
                                                                                                                    train_loss,
                                                                                                                    test_loss,
                                                                                                                    train_acc,
                                                                                                                    test_acc))
            if train_acc > best_accuracy:
                saveModel()
                best_accuracy = train_acc
                print("Best Accuracy:{} %".format(best_accuracy))

            history['train_loss'].append(train_loss)
            history['test_loss'].append(test_loss)
            history['train_acc'].append(train_acc)
            history['test_acc'].append(test_acc)   

    df_history = pd.DataFrame(data=history)
    df_history.to_csv("historic.csv", encoding='utf-8', index=False)

## Training

In [12]:
train(10)
print('Finished Training')

Fold 1
Epoch:1/10 AVG Training Loss:3.786 AVG Test Loss:1.932 AVG Training Acc 23.88 % AVG Test Acc 18.56 %
Best Accuracy:23.884610683290553 %
Epoch:2/10 AVG Training Loss:2.450 AVG Test Loss:1.950 AVG Training Acc 26.98 % AVG Test Acc 18.56 %
Best Accuracy:26.980198019801982 %
Epoch:3/10 AVG Training Loss:22313.729 AVG Test Loss:1.971 AVG Training Acc 27.02 % AVG Test Acc 18.56 %
Best Accuracy:27.019924214643687 %
Epoch:4/10 AVG Training Loss:13127.991 AVG Test Loss:1.986 AVG Training Acc 27.51 % AVG Test Acc 18.56 %
Best Accuracy:27.511917858452513 %
Epoch:5/10 AVG Training Loss:20394.231 AVG Test Loss:1.996 AVG Training Acc 27.27 % AVG Test Acc 18.56 %
Epoch:6/10 AVG Training Loss:4136.192 AVG Test Loss:2.004 AVG Training Acc 27.45 % AVG Test Acc 18.56 %
Epoch:7/10 AVG Training Loss:1105.620 AVG Test Loss:2.010 AVG Training Acc 27.30 % AVG Test Acc 18.56 %
Epoch:8/10 AVG Training Loss:1003.187 AVG Test Loss:2.009 AVG Training Acc 27.35 % AVG Test Acc 18.56 %
Epoch:9/10 AVG Training 